In [ ]:
import json
import pandas as pd
import os

df_test = pd.read_csv("vuln_data_test.csv")

SYSTEM_PROMPT = """You are the smartest AI solidity smart contract security auditor in the world."""

PROMPTS = """The given Solidity function is vulnerable. Review the code and analyze its security flaws. Just give short explanation why these function is vulnerable.
    
    This is the function we need to audit:
    ```solidity
    {code}
    ```
    
Vulnerability : """

df_test['input_prompt'] = df_test['vuln_code'].map(lambda x: PROMPTS.format(code=x))

# Parameters
max_tokens_per_batch = 60000  # Set below the 900,000 token limit to be safe
estimated_tokens_per_request = 1000  # Adjust based on your average request size
requests_per_batch = max_tokens_per_batch // estimated_tokens_per_request

# Create output directory
output_dir = "batch_input_zeroshot_4o"
os.makedirs(output_dir, exist_ok=True)

batch_count = 0
request_count = 0
batch_file = open(os.path.join(output_dir, f"batch_input_{batch_count}.jsonl"), "w")

for index, row in df_test.iterrows():
    user_prompt = row['input_prompt']

    task = {
        "custom_id": f"request-{index}",
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model": "gpt-4o",
            "temperature": 0.1,
            "messages": [
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": user_prompt}
            ]
        }
    }

    batch_file.write(json.dumps(task) + "\n")
    request_count += 1

    if request_count >= requests_per_batch:
        batch_file.close()
        batch_count += 1
        request_count = 0
        batch_file = open(os.path.join(output_dir, f"batch_input_{batch_count}.jsonl"), "w")

batch_file.close()

In [ ]:
# Read the original CSV
df = pd.read_csv("vuln_data_test.csv")

# Path to your batch output directory
batch_output_dir = "batch_outputs_zeroshot_4o"

# Gather all output lines from multiple batch files
output_lines = []
for batch_idx in range(len(os.listdir(batch_output_dir))):
    file_path = os.path.join(batch_output_dir, f"batch_output_{batch_idx}.jsonl")
    with open(file_path) as f:
        output_lines.extend(f.readlines())

# Process each output line using custom_id
for line in output_lines:
    try:
        json_out = json.loads(line)
        custom_id = json_out.get("custom_id", "")
        string = json_out['response']['body']['choices'][0]['message']['content']

        # Extract original index from custom_id like "request-123"
        original_index = int(custom_id.split("-")[-1])

        ground_truth = df_test[df_test['vuln_code'] == df.loc[original_index, 'vuln_code']]['full_context'].values[0]

        df.loc[original_index, "llm_output"] = string
        df.loc[original_index, "ground_truth"] = ground_truth

    except (json.JSONDecodeError, KeyError, ValueError) as e:
        print(f"Error processing custom_id {custom_id}: {e}")
        print(f"Raw response: {line}")

In [40]:
df.to_csv("zeroshot_gpt4o.csv")